TODO: Add evaluation

In [71]:
import pandas as pd 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
#%matplotlib notebook

from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


In [72]:
# datasets provided by https://github.com/llSourcell/recommender_live
dataset1 = '/home/kristina/Downloads/10000.txt'
dataset2 = '/home/kristina/Downloads/song_data.csv'

temp_df1 = pd.read_csv(dataset1, sep="\t", names =["user_id", "song_id", "listen_count"])
temp_df2 = pd.read_csv(dataset2)

final_df = pd.merge(temp_df1, temp_df2)
final_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,930d2be6c85315d72cab9823ec0f7bfe7e477794,SOBBMDR12A8C13253B,1.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007


In [73]:
train_data, test_data = cv.train_test_split(final_df, test_size=0.25)

In [74]:
subset_df = train_data[['user_id','song_id', 'listen_count']]
matrix_df = pd.crosstab(subset_df.user_id, subset_df.song_id, values = subset_df.listen_count, aggfunc="first")
matrix_df.fillna(0, inplace=True)

magnitude = np.sqrt(np.square(matrix_df).sum(axis=1))
matrix_df = matrix_df.divide(magnitude, axis='index')

data_sparse = sparse.csr_matrix(matrix_df)
similarities = cosine_similarity(data_sparse.transpose())
sim = pd.DataFrame(data=similarities, index=matrix_df.columns, columns= matrix_df.columns)
sim.head()

song_id,SOAAAGQ12A8C1420C8,SOAAFAC12A67ADF7EB,SOAAOYI12AB01831CE,SOAARXR12A8C133D15,SOAATLI12A8C13E319,SOAAUKC12AB017F868,SOAAVUV12AB0186646,SOAAWEE12A6D4FBEC8,SOABHYV12A6D4F6D0F,SOABJBU12A8C13F63F,...,SOZXQRQ12A6701C323,SOZYBGN12A8C13A93C,SOZYDZR12A8C13F4F0,SOZYPRF12A6310F1B4,SOZYSDT12A8C13BFD7,SOZYUGZ12A8AE472AC,SOZZIOH12A67ADE300,SOZZLZN12A8AE48D6D,SOZZRHE12A6702165F,SOZZYAO12A6701FF36
song_id,,,,,,,,,,,,,,,,,,,,,
SOAAAGQ12A8C1420C8,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAAFAC12A67ADF7EB,0.0,1.0,0.0,0.0,0.0,0.0,0.036637,0.0,0.0,0.123373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAAOYI12AB01831CE,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAARXR12A8C133D15,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAATLI12A8C13E319,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
#item-item collaboration
song_id_ds = 'SOBXHDL12A81C204C0'
data = sim.loc[song_id_ds].nlargest(50)
df1 = final_df.loc[final_df['song_id'].isin(data.index.values)]
df1 = df1.drop_duplicates(subset=['song_id'], keep="first")
print(len(df1))
print("Song ID :" + str(song_id_ds))
df1.head(n = 10)

50
Song ID :SOBXHDL12A81C204C0


,user_id,song_id,listen_count,title,release,artist_name,year
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007
945,b64cdd1a0bd907e5e00b39e345194768e330d652,SOALEQA12A58A77839,1.0,Jumping Jack Flash,Shine A Light,The Rolling Stones,1982
1454,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOCAHRT12A8C13A1A4,5.0,S.O.S.,SOS,Jonas Brothers,2007
3270,bd8475385f0aa78830fa6dfce9e7242164b035c8,SOSDTGC12AF72A2BC3,1.0,Chinese,It's Not Me_ It's You,Lily Allen,2009
4151,a58de017cbeda1763ea002fe027ed41b4ed53109,SOTNEEA12A6D4F7F8D,1.0,Born On The Bayou,At The Movies,Creedence Clearwater Revival,1969
4174,a58de017cbeda1763ea002fe027ed41b4ed53109,SOUZLXN12A6D4F4A66,1.0,Back To Black,Back To Black,Amy Winehouse,2006
4977,99b5f915dd56a0150eb91605e1f6109e1e80c9de,SOKAJXI12AB017DEC7,1.0,Brimstone Landscapes,Fragmentary Evidence,Augury,2009
5337,732f88be38fae217f8ab7e24c20dd072436e3e40,SOMUWFQ12AB0184608,1.0,Lookin' Out My Back Door,The Singles Collection,Creedence Clearwater Revival,1970
5596,1a849df9dabb15845eb932d46d81e2fd77176786,SOWSEAO12A58A7B3EA,2.0,Everybody Loves Me,Waking Up,OneRepublic,2009
5665,daebcdcf97caaf54a327b8ff52eca9f320599a10,SOJEBJO12A6701C322,1.0,Cry,80s Triple Set,Godley & Creme,1985


In [76]:
final_df["user_id"] = pd.Categorical(final_df["user_id"])
final_df['user_id'] = final_df["user_id"].cat.codes

final_df["song_id"] = pd.Categorical(final_df["song_id"])
final_df['song_id'] = final_df["song_id"].cat.codes

final_df["title"] = pd.Categorical(final_df["title"])
final_df['title'] = final_df["title"].cat.codes

final_df["release"] = pd.Categorical(final_df["release"])
final_df['release'] = final_df["release"].cat.codes

final_df["artist_name"] = pd.Categorical(final_df["artist_name"])
final_df['artist_name'] = final_df["artist_name"].cat.codes

final_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,278,78,1.0,4404,2929,847,0
1,278,233,2.0,1272,946,1385,1976
2,219,233,1.0,1272,946,1385,1976
3,278,414,1.0,4169,1065,982,2007
4,278,414,1.0,4169,1065,982,2007


In [77]:
train_data, test_data = cv.train_test_split(final_df, test_size=0.25)
df = final_df[['user_id', 'song_id', 'listen_count']]
#df = final_df.loc[cluster_map["Cluster Class"]== 4]

n_users = df.user_id.unique().shape[0]
n_items = df.song_id.unique().shape[0]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

subset_df_test = test_data[['user_id','song_id', 'listen_count']]
matrix_df_test = pd.crosstab(subset_df_test.user_id, subset_df_test.song_id, values = subset_df_test.listen_count, aggfunc="first")
matrix_df_test.fillna(0, inplace=True)

magnitude_test = np.sqrt(np.square(matrix_df_test).sum(axis=1))
matrix_df_test = matrix_df_test.divide(magnitude_test, axis='index')

data_sparse_test = sparse.csr_matrix(matrix_df_test)
similarities_test = cosine_similarity(data_sparse_test.transpose())


In [78]:

rmse_user = similarities[similarities_test.nonzero()].flatten()
rmse_test = similarities_test[similarities_test.nonzero()].flatten()
result = sqrt(mean_squared_error(rmse_user, rmse_test))

print ('Evaluation for User Collaboration: ' + str(result))
print('Result represents "Error regression loss" - a perfect result is 0')

Evaluation for User Collaboration: 0.7131250020762674
Result represents "Error regression loss" - a perfect result is 0
